<a href="https://colab.research.google.com/github/marekk1717/letsautomate/blob/master/Commvault_Python_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of cvpysdk and required dependencies

In [0]:
!pip install xmltodict future requests
!pip install git+https://github.com/CommvaultEngg/cvpysdk.git@dev

In [0]:
from cvpysdk.commcell import Commcell

jj

In [0]:
WEBCONSOLE_URL = 'democommserve.commvault.com'
COMMCELL_USERNAME = 'mkedzierski'
COMMCELL_PASSWORD = 'Commvault!2'

Connect to [WebConsole](https://democommserve.commvault.com)

In [10]:
commcell = Commcell(webconsole_hostname=WEBCONSOLE_URL, commcell_username=COMMCELL_USERNAME, commcell_password=COMMCELL_PASSWORD)
print('Connected to ' + commcell.commserv_hostname + ', Version: v11SP' + str(commcell.commserv_version))

Connected to COMMSERVE.cv.lab, Version: v11SP18
